In [0]:
%sql
USE CATALOG reussma;
USE SCHEMA fsi_genie;

In [0]:
import pandas as pd

customers = pd.read_csv("https://github.com/marianreuss/fsi-genie/raw/refs/heads/main/customers_sampled.csv")
customers = spark.createDataFrame(customers)
customers.display()
customers.write.mode("overwrite").table("customers")

point_of_contact,telephone_number,email,customer_since,ticker
Donnie Winters,14801340,donnie.winters@agrify.com,2006-06-06,agfy
Masha Spires,18830012,masha.spires@cummins.com,2021-12-13,cmi
Shandeigh Estrella,18547681,shandeigh.estrella@altramotion.com,2022-07-15,aimc
Samara Harter,15281200,sharter@unither.com,2017-10-07,uthr
Kelcie Cazares,15775181,kelcie.cazares@dovercorporation.com,2014-06-29,dov
Amabelle Magee,17003411,a.magee@acmrcsh.com,2006-11-29,acmr
Florrie Westbrook,1864341274,f.westbrook@amciacquisition.com,2022-04-28,adn
Bel Wells,1263465760,b.wells@fastenal.com,2020-04-29,fast
Talyah Faison,1638640001,tfaison@organon.com,2018-10-25,ogn
Lauryn Troutman,1516447030,ltroutman@monsterbevcorp.com,2013-12-31,mnst


In [0]:
spark.table("data_2").display()

ticker,name,currency,exchange,industry,logo,weburl,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,last_processing_date,total_grade,total_level,cik
dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/ef50b4a2b263c847211b567a01edb702cae8b9ef46ca1f6856ca4432a70e9f52.png,https://thewaltdisneycompany.com/,A,High,BB,Medium,BB,Medium,510,316,321,1147,2022-04-19,BBB,High,1744489
gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,https://static.finnhub.io/logo/9253db78-80c9-11ea-920d-00000000092a.png,https://www.gm.com/,A,High,BB,Medium,B,Medium,510,303,255,1068,2022-04-17,BBB,High,1467858
gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,https://static.finnhub.io/logo/f153dcda-80eb-11ea-bc2e-00000000092a.png,https://www.grainger.com/,B,Medium,BB,Medium,B,Medium,255,385,240,880,2022-04-19,BB,Medium,277135
mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/26868a62-80ec-11ea-9712-00000000092a.png,https://mohawkind.com/,A,High,B,Medium,BB,Medium,570,298,303,1171,2022-04-18,BBB,High,851968
lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,https://static.finnhub.io/logo/1cd144d2-80ec-11ea-b50a-00000000092a.png,https://www.livenationentertainment.com/,BBB,High,BB,Medium,B,Medium,492,310,250,1052,2022-04-18,BBB,High,1335258
lvs,Las Vegas Sands Corp,USD,"NEW YORK STOCK EXCHANGE, INC.",Hotels Restaurants and Leisure,https://static.finnhub.io/logo/7256a9be-8279-11ea-874d-00000000092a.png,https://www.sands.com/,A,High,BB,Medium,BB,Medium,547,318,313,1178,2022-04-18,BBB,High,1300514
clx,Clorox Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,https://static.finnhub.io/logo/a67c34fe-80eb-11ea-a87f-00000000092a.png,https://www.thecloroxcompany.com/,A,High,BB,Medium,BB,Medium,560,350,345,1255,2022-04-16,A,High,21076
aacg,ATA Creativity Global,CNY,NASDAQ NMS - GLOBAL MARKET,Diversified Consumer Services,https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AACG.svg,http://www.ata.net.cn,B,Medium,B,Medium,B,Medium,203,200,205,608,2022-07-01,B,Medium,1420529
aal,American Airlines Group Inc,USD,NASDAQ NMS - GLOBAL MARKET,Airlines,https://static2.finnhub.io/file/publicdatany/finnhubimage/stock_logo/AAL.png,https://americanairlines.gcs-web.com/,B,Medium,B,Medium,B,Medium,270,211,265,746,2022-04-16,B,Medium,6201
aame,Atlantic American Corp,USD,NASDAQ NMS - GLOBAL MARKET,Insurance,https://static.finnhub.io/logo/6b536b3e-826a-11ea-bc66-00000000092a.png,http://www.atlam.com/,B,Medium,B,Medium,BB,Medium,220,221,300,741,2022-04-16,B,Medium,8177


In [0]:
import random
import pyspark.sql.functions as F

esg = spark.table("data_2").filter("weburl is not null")
esg = esg.toPandas()
esg2 = esg.copy(deep=True)
cols = ['ticker',
 'name',
 'currency',
 'exchange',
 'industry',
 ]


grades = ['environment_grade',
 'environment_level',
 'social_grade',
 'social_level',
 'governance_grade',
 'governance_level',
 'environment_score',
 'social_score',
 'governance_score',
 'total_score',
 'total_grade',
 'total_level',
 'cik']


esg2 = esg2[cols]
esg2

for g in grades:
  esg2[g] = esg2["ticker"].apply(lambda _: random.choice(esg[g].unique().tolist()))

esg2

,ticker,name,currency,exchange,industry,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,total_grade,total_level,cik
0,dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,Medium,AA,Medium,BBB,Low,299,307,248,1528,BBB,High,80420
1,gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,B,Excellent,BB,Excellent,C,Low,270,313,348,1126,B,High,927003
2,gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,BB,Medium,AA,Medium,C,High,350,231,321,897,A,High,1725255
3,mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,BBB,Excellent,BB,Low,B,Medium,242,219,223,914,BBB,High,1467373
4,lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,High,BBB,Excellent,BBB,Medium,558,394,240,654,BBB,High,1385157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,hi,Hillenbrand Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Machinery,BBB,Medium,CCC,Medium,B,High,527,364,290,1115,BBB,Medium,946581
702,hhc,Howard Hughes Corp,USD,"NEW YORK STOCK EXCHANGE, INC.",Real Estate,AA,High,BBB,Medium,BB,High,675,334,325,1104,B,Medium,217346
703,hubs,HubSpot Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Technology,B,Excellent,B,Low,C,Low,307,296,370,1267,B,Medium,798354
704,hesm,Hess Midstream LP,USD,"NEW YORK STOCK EXCHANGE, INC.",Energy,B,High,BB,Low,B,Low,217,218,350,1127,B,High,1018724


In [0]:
%sql
DROP TABLE esg

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1524276377181585>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'DROP TABLE esg\n')

File /databricks/python/lib/python3.11/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:153, in SqlMagic.sql(self, line, cell)
    150 if self.use_py4j:
    151     return self.execute_via_py4j(cell)
--> 153 return self.execu

In [0]:
spark.createDataFrame(esg2).write.mode("overwrite").saveAsTable("esg")

In [0]:
spark.createDataFrame(esg2).display()

ticker,name,currency,exchange,industry,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,total_grade,total_level,cik
dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,Medium,AA,Medium,BBB,Low,299,307,248,1528,BBB,High,80420
gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,B,Excellent,BB,Excellent,C,Low,270,313,348,1126,B,High,927003
gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,BB,Medium,AA,Medium,C,High,350,231,321,897,A,High,1725255
mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,BBB,Excellent,BB,Low,B,Medium,242,219,223,914,BBB,High,1467373
lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,High,BBB,Excellent,BBB,Medium,558,394,240,654,BBB,High,1385157
lvs,Las Vegas Sands Corp,USD,"NEW YORK STOCK EXCHANGE, INC.",Hotels Restaurants and Leisure,B,High,BB,Low,B,Low,668,259,355,1067,A,High,20286
clx,Clorox Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,BBB,Medium,B,High,BB,Low,363,360,285,1009,BB,High,1324424
aacg,ATA Creativity Global,CNY,NASDAQ NMS - GLOBAL MARKET,Diversified Consumer Services,BB,Excellent,B,High,C,High,492,253,255,759,BBB,High,1159036
aal,American Airlines Group Inc,USD,NASDAQ NMS - GLOBAL MARKET,Airlines,BBB,Medium,B,High,BBB,Low,238,324,358,1007,BB,High,1300514
aame,Atlantic American Corp,USD,NASDAQ NMS - GLOBAL MARKET,Insurance,B,Excellent,A,Medium,BB,High,275,395,400,812,BB,Medium,1158449


In [0]:
spark.table("reussma.fsi_genie.esg").columns

['ticker',
 'name',
 'currency',
 'exchange',
 'industry',
 'environment_grade',
 'environment_level',
 'social_grade',
 'social_level',
 'governance_grade',
 'governance_level',
 'environment_score',
 'social_score',
 'governance_score',
 'total_score',
 'total_grade',
 'total_level',
 'cik']

In [0]:
%sql
select * from reussma.fsi_genie.esg

ticker,name,currency,exchange,industry,environment_grade,environment_level,social_grade,social_level,governance_grade,governance_level,environment_score,social_score,governance_score,total_score,total_grade,total_level,cik
dis,Walt Disney Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,Medium,AA,Medium,BBB,Low,299,307,248,1528,BBB,High,80420
gm,General Motors Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Automobiles,B,Excellent,BB,Excellent,C,Low,270,313,348,1126,B,High,927003
gww,WW Grainger Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Trading Companies and Distributors,BB,Medium,AA,Medium,C,High,350,231,321,897,A,High,1725255
mhk,Mohawk Industries Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,BBB,Excellent,BB,Low,B,Medium,242,219,223,914,BBB,High,1467373
lyv,Live Nation Entertainment Inc,USD,"NEW YORK STOCK EXCHANGE, INC.",Media,BB,High,BBB,Excellent,BBB,Medium,558,394,240,654,BBB,High,1385157
lvs,Las Vegas Sands Corp,USD,"NEW YORK STOCK EXCHANGE, INC.",Hotels Restaurants and Leisure,B,High,BB,Low,B,Low,668,259,355,1067,A,High,20286
clx,Clorox Co,USD,"NEW YORK STOCK EXCHANGE, INC.",Consumer products,BBB,Medium,B,High,BB,Low,363,360,285,1009,BB,High,1324424
aacg,ATA Creativity Global,CNY,NASDAQ NMS - GLOBAL MARKET,Diversified Consumer Services,BB,Excellent,B,High,C,High,492,253,255,759,BBB,High,1159036
aal,American Airlines Group Inc,USD,NASDAQ NMS - GLOBAL MARKET,Airlines,BBB,Medium,B,High,BBB,Low,238,324,358,1007,BB,High,1300514
aame,Atlantic American Corp,USD,NASDAQ NMS - GLOBAL MARKET,Insurance,B,Excellent,A,Medium,BB,High,275,395,400,812,BB,Medium,1158449


In [0]:
%sql
select * from reussma.fsi_genie.customers

point_of_contact,telephone_number,email,customer_since,ticker
Donnie Winters,+14801340,donnie.winters@agrify.com,2006-06-06,agfy
Masha Spires,+18830012,masha.spires@cummins.com,2021-12-13,cmi
Shandeigh Estrella,+18547681,shandeigh.estrella@altramotion.com,2022-07-15,aimc
Samara Harter,+15281200,sharter@unither.com,2017-10-07,uthr
Kelcie Cazares,+15775181,kelcie.cazares@dovercorporation.com,2014-06-29,dov
Amabelle Magee,+17003411,a.magee@acmrcsh.com,2006-11-29,acmr
Florrie Westbrook,+1864341274,f.westbrook@amciacquisition.com,2022-04-28,adn
Bel Wells,+1263465760,b.wells@fastenal.com,2020-04-29,fast
Talyah Faison,+1638640001,tfaison@organon.com,2018-10-25,ogn
Lauryn Troutman,+1516447030,ltroutman@monsterbevcorp.com,2013-12-31,mnst
